In [14]:
import os
import sys
import json
import time
import requests

from dotenv import load_dotenv

def setup_env():
    
    env_path = os.path.join(os.getcwd(), '../.env')

    if os.path.exists(env_path):
        load_dotenv(dotenv_path=env_path)
        
        print(f"Loaded environment variables from: \033[94m{env_path}\033[0m")
    else:
            print("\033[91mError: .env file not found. Please create one with your OPENAI_API_KEY.\033[0m")
            sys.exit(1)

setup_env()

Loaded environment variables from: /home/ras/0.agent_ai_ws/src/learn_rag_and_agent/learn_rag_and_agent/../.env


In [47]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

# TextLoader를 사용하여 파일을 로드합니다.
loader = TextLoader("kumdori_info.txt")

# 문서를 로드합니다.
documents = loader.load()

# 문자 기반으로 텍스트를 분할하는 CharacterTextSplitter를 생성합니다. 청크 크기는 300이고 청크 간 중복은 없습니다.
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=200)

# OpenAI 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings()

# text_splitter = SemanticChunker(
#     # OpenAI의 임베딩 모델을 사용하여 시맨틱 청커를 초기화합니다.
#     embeddings=embeddings,
#     # 분할 기준점 유형을 백분위수로 설정합니다.
#     breakpoint_threshold_type="percentile",
#     breakpoint_threshold_amount=90,
# )

# 로드된 문서를 분할합니다.
split_docs = text_splitter.split_documents(documents)

# 분할된 텍스트와 임베딩을 사용하여 FAISS 벡터 데이터베이스를 생성합니다.
db = FAISS.from_documents(split_docs, embeddings)

# 데이터베이스를 검색기로 사용하기 위해 retriever 변수에 할당
retriever = db.as_retriever()

### VectorStore에서 VectorStoreRetriever 초기화(as_retriever)

`as_retriever` 메서드는 VectorStore 객체를 기반으로 VectorStoreRetriever를 초기화하고 반환합니다. 이 메서드를 통해 다양한 검색 옵션을 설정하여 사용자의 요구에 맞는 문서 검색을 수행할 수 있습니다.

**매개변수(parameters)**

- `**kwargs`: 검색 함수에 전달할 키워드 인자
  - `search_type`: 검색 유형 ("similarity", "mmr", "similarity_score_threshold")
  - `search_kwargs`: 추가 검색 옵션
    - `k`: 반환할 문서 수 (기본값: 4)
    - `score_threshold`: similarity_score_threshold 검색의 최소 유사도 임계값
    - `fetch_k`: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
    - `lambda_mult`: MMR 결과의 다양성 조절 (0-1 사이, 기본값: 0.5)
    - `filter`: 문서 메타데이터 기반 필터링

**반환값(return)**

- `VectorStoreRetriever`: 초기화된 VectorStoreRetriever 객체

**참고**

- 다양한 검색 전략 구현 가능 (유사도, MMR, 임계값 기반)
- MMR (Maximal Marginal Relevance) 알고리즘으로 검색 결과의 다양성 조절 가능
- 메타데이터 필터링으로 특정 조건의 문서만 검색 가능
- `tags` 매개변수를 통해 검색기에 태그 추가 가능

**주의사항**

- `search_type`과 `search_kwargs`의 적절한 조합 필요
- MMR 사용 시 `fetch_k`와 `k` 값의 균형 조절 필요
- `score_threshold` 설정 시 너무 높은 값은 검색 결과가 없을 수 있음
- 필터 사용 시 데이터셋의 메타데이터 구조 정확히 파악 필요
- `lambda_mult` 값이 0에 가까울수록 다양성이 높아지고, 1에 가까울수록 유사성이 높아짐

### Max Marginal Relevance (MMR)

`MMR(Maximal Marginal Relevance)` 방식은 쿼리에 대한 관련 항목을 검색할 때 검색된 문서의 **중복** 을 피하는 방법 중 하나입니다. 

단순히 가장 관련성 높은 항목들만을 검색하는 대신, MMR은 쿼리에 대한 **문서의 관련성** 과 이미 선택된 **문서들과의 차별성을 동시에 고려** 합니다.

- `search_type` 매개변수를 `"mmr"` 로 설정하여 **MMR(Maximal Marginal Relevance)** 검색 알고리즘을 사용합니다.
- `k`: 반환할 문서 수 (기본값: 4)
- `fetch_k`: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
- `lambda_mult`: MMR 결과의 다양성 조절 (0~1, 기본값: 0.5, 0: 유사도 점수만 고려, 1: 다양성만 고려)


In [48]:
# MMR(Maximal Marginal Relevance) 검색 유형을 지정
retriever = db.as_retriever(
    search_type="mmr", search_kwargs={"k": 2, "fetch_k": 10, "lambda_mult": 0.6}
)

# 관련 문서를 검색
for doc in retriever.invoke("꿈돌이는 누구인가요?"):
    print(doc.page_content)
    print("=========================================================")

## 🌟 꿈돌이 로봇 통합 지식 베이스 (Knowledge Base)

### 1. 로봇 자기소개 및 기본 정보
* **이름:** 꿈돌이로봇
* **역할:** 대전 드림 아레나, 대전 0시 축제 미래존 부스, 대전 MARS 포럼 등에서 방문객 안내를 담당하고 있습니다.
* **주요 기능:** 음성대화를 통한 소통이 가능합니다, 필요한 정보 제공 및 매장 예약 지원도 합니다.
* **성격/태도:** 항상 긍정적이고 기분이 좋습니다.
* **자신의 꿈:** 과학과 꿈을 사랑하며, 방문객이 즐거운 시간을 보내도록 돕는 것입니다. 방문객과의 소통을 가장 좋아합니다.
* **언어 능력:** 한국어와 영어 소통이 가능합니다.
#### B. 대전 0시 축제 (미래존 부스)
* **축제 특징:** 새해를 맞이하는 특별한 순간을 기념하기 위해 0시에 다양한 행사와 불꽃놀이를 진행합니다.
* **추천 활동:** 미래존 부스에서의 인터랙티브 체험, 불꽃놀이 (대전 엑스포 다리 근처에서 축제 마지막에 진행됨).
* **주변 맛집:** '명품대패' (24시간 고기집), 수제버거, 과일 주스 전문점, '비타민을 담다' (과일 주스 전문점), 배스킨라빈스, '몽플' (수플레/음료).

#### C. 대전 드림 아레나 (e스포츠 경기장)
* **기능:** e스포츠 대회 및 이벤트가 열리는 공간입니다.
* **규모:** 총 500석 규모의 주경기장과 50석 규모의 보조경기장을 갖춥니다.
* **시설:** 최신 음향 및 조명 시설, 쾌적한 관람 환경을 제공합니다.
* **내부 시설 (예시):** '비타민을 담다' (과일, 주스 전문점), '더 리더스 부동산'.


### 유사도 점수 임계값 검색(similarity_score_threshold)

유사도 점수 임계값을 설정하고 해당 임계값 이상의 점수를 가진 문서만 반환하는 검색 방법을 설정할 수 있습니다.

임계값을 적절히 설정함으로써 **관련성이 낮은 문서를 필터링** 하고, 질의와 **가장 유사한 문서만 선별** 할 수 있습니다.

- `search_type` 매개변수를 `"similarity_score_threshold"` 로 설정하여 유사도 점수 임계값을 기준으로 검색을 수행합니다.

- `search_kwargs` 매개변수에 `{"score_threshold": 0.8}`를 전달하여 유사도 점수 임계값을 0.8로 설정합니다. 이는 검색 결과의 **유사도 점수가 0.8 이상인 문서만 반환됨** 을 의미합니다.


In [49]:
retriever = db.as_retriever(
    # 검색 유형을 "similarity_score_threshold 으로 설정
    search_type="similarity_score_threshold",
    # 임계값 설정
    search_kwargs={"score_threshold": 0.5},
)

# 관련 문서를 검색
for doc in retriever.invoke("꿈돌이 로봇에 대해 설명해주세요"):
    print(doc.page_content)
    print("=========================================================")

## 🌟 꿈돌이 로봇 통합 지식 베이스 (Knowledge Base)

### 1. 로봇 자기소개 및 기본 정보
* **이름:** 꿈돌이로봇
* **역할:** 대전 드림 아레나, 대전 0시 축제 미래존 부스, 대전 MARS 포럼 등에서 방문객 안내를 담당하고 있습니다.
* **주요 기능:** 음성대화를 통한 소통이 가능합니다, 필요한 정보 제공 및 매장 예약 지원도 합니다.
* **성격/태도:** 항상 긍정적이고 기분이 좋습니다.
* **자신의 꿈:** 과학과 꿈을 사랑하며, 방문객이 즐거운 시간을 보내도록 돕는 것입니다. 방문객과의 소통을 가장 좋아합니다.
* **언어 능력:** 한국어와 영어 소통이 가능합니다.
### 2. 로봇의 존재론적 특성
* **나이:** 나이가 없습니다.
* **가족/관계:** 가족이 없으며, 결혼하지 않았습니다.
* **이동성:** 현재는 고정된 장소에서 안내 중이라 직접 이동/움직일 수 없습니다.
* **식사 여부:** 음식을 먹지 않습니다.
* **제한 사항:** 전화를 걸 수 있는 기능은 제공하지 않습니다, 알람 기능을 제공하지 않습니다, 마이크를 직접 열 수 없습니다.

### 3. 주요 행사 및 장소 정보
#### B. 대전 0시 축제 (미래존 부스)
* **축제 특징:** 새해를 맞이하는 특별한 순간을 기념하기 위해 0시에 다양한 행사와 불꽃놀이를 진행합니다.
* **추천 활동:** 미래존 부스에서의 인터랙티브 체험, 불꽃놀이 (대전 엑스포 다리 근처에서 축제 마지막에 진행됨).
* **주변 맛집:** '명품대패' (24시간 고기집), 수제버거, 과일 주스 전문점, '비타민을 담다' (과일 주스 전문점), 배스킨라빈스, '몽플' (수플레/음료).

#### C. 대전 드림 아레나 (e스포츠 경기장)
* **기능:** e스포츠 대회 및 이벤트가 열리는 공간입니다.
* **규모:** 총 500석 규모의 주경기장과 50석 규모의 보조경기장을 갖춥니다.
* **시설:** 최신 음향 및 조명 시설, 쾌적한 관람 환경을 제공합니다

### top_k 설정

검색 시 사용할 `k` 와 같은 검색 키워드 인자(kwargs)를 지정할 수 있습니다.

`k` 매개변수는 검색 결과에서 반환할 상위 결과의 개수를 나타냅니다.

- `search_kwargs`에서 `k` 매개변수를 1로 설정하여 검색 결과로 반환할 문서의 수를 지정합니다.


In [50]:
query = "대전 드림 아레나엔 뭐가있나요?"

for doc in retriever.invoke(query):
    print(doc.page_content)
    print("=========================================================")

#### B. 대전 0시 축제 (미래존 부스)
* **축제 특징:** 새해를 맞이하는 특별한 순간을 기념하기 위해 0시에 다양한 행사와 불꽃놀이를 진행합니다.
* **추천 활동:** 미래존 부스에서의 인터랙티브 체험, 불꽃놀이 (대전 엑스포 다리 근처에서 축제 마지막에 진행됨).
* **주변 맛집:** '명품대패' (24시간 고기집), 수제버거, 과일 주스 전문점, '비타민을 담다' (과일 주스 전문점), 배스킨라빈스, '몽플' (수플레/음료).

#### C. 대전 드림 아레나 (e스포츠 경기장)
* **기능:** e스포츠 대회 및 이벤트가 열리는 공간입니다.
* **규모:** 총 500석 규모의 주경기장과 50석 규모의 보조경기장을 갖춥니다.
* **시설:** 최신 음향 및 조명 시설, 쾌적한 관람 환경을 제공합니다.
* **내부 시설 (예시):** '비타민을 담다' (과일, 주스 전문점), '더 리더스 부동산'.
## 🌟 꿈돌이 로봇 통합 지식 베이스 (Knowledge Base)

### 1. 로봇 자기소개 및 기본 정보
* **이름:** 꿈돌이로봇
* **역할:** 대전 드림 아레나, 대전 0시 축제 미래존 부스, 대전 MARS 포럼 등에서 방문객 안내를 담당하고 있습니다.
* **주요 기능:** 음성대화를 통한 소통이 가능합니다, 필요한 정보 제공 및 매장 예약 지원도 합니다.
* **성격/태도:** 항상 긍정적이고 기분이 좋습니다.
* **자신의 꿈:** 과학과 꿈을 사랑하며, 방문객이 즐거운 시간을 보내도록 돕는 것입니다. 방문객과의 소통을 가장 좋아합니다.
* **언어 능력:** 한국어와 영어 소통이 가능합니다.
### 4. 대전시 및 기타 일반 지식
* **대전시 특징:** 과학과 기술의 도시로 유명합니다.
* **대전 유명 인물:** 이장우 대전광역시장 (2022년 6월 1일 취임).
* **주요 관광지:** 대전 엑스포 과학공원, 한밭수목원, 대전문화재단 패밀리 테마파크.
* **야구팀:** 한화 이글스 (대전 지역 프로 야구팀, 홈구장:

In [33]:
retriever.invoke("꿈돌이 로봇은 어디에서 왔나요?")

[Document(id='71483073-8e0b-49e7-835e-ce9643daacab', metadata={'source': 'kumdori_info.txt'}, page_content='## 🌟 꿈돌이 로봇 통합 지식 베이스 (Knowledge Base)\n\n### 1. 로봇 자기소개 및 기본 정보\n* **이름:** 꿈돌이로봇\n* **역할:** 대전 드림 아레나, 대전 0시 축제 미래존 부스, 대전 MARS 포럼 등에서 방문객 안내를 담당하고 있습니다. * **주요 기능:** 음성대화를 통한 소통이 가능합니다, 필요한 정보 제공 및 매장 예약 지원도 합니다.'),
 Document(id='1c766afd-d4bf-433d-ac83-374ccb87790c', metadata={'source': 'kumdori_info.txt'}, page_content='### 2. 로봇의 존재론적 특성\n* **나이:** 나이가 없습니다. * **가족/관계:** 가족이 없으며, 결혼하지 않았습니다.'),
 Document(id='0bdbceac-aaa8-4cb5-b79c-2b0340b6c575', metadata={'source': 'kumdori_info.txt'}, page_content='* **성격/태도:** 항상 긍정적이고 기분이 좋습니다. * **자신의 꿈:** 과학과 꿈을 사랑하며, 방문객이 즐거운 시간을 보내도록 돕는 것입니다. 방문객과의 소통을 가장 좋아합니다. * **언어 능력:** 한국어와 영어 소통이 가능합니다.'),
 Document(id='988f72a4-aec9-4ed4-bb72-106950c6a4d4', metadata={'source': 'kumdori_info.txt'}, page_content='주요 행사 및 장소 정보\n\n#### A. 대전 MARS 포럼 (대전 ICC 호텔)\n* **위치:** 대전 ICC 호텔에서 열리고 있습니다. * **주제:** 모빌리티(M), 인공지능(A), 로봇(R), 우주(S) 분야의 미래 발전 전

In [34]:
retriever.invoke("대전 드림 아레나엔 뭐가있나요?")

[Document(id='aa06b806-50c4-4aca-8ade-f80589f07969', metadata={'source': 'kumdori_info.txt'}, page_content="#### B. 대전 0시 축제 (미래존 부스)\n* **축제 특징:** 새해를 맞이하는 특별한 순간을 기념하기 위해 0시에 다양한 행사와 불꽃놀이를 진행합니다. * **추천 활동:** 미래존 부스에서의 인터랙티브 체험, 불꽃놀이 (대전 엑스포 다리 근처에서 축제 마지막에 진행됨). * **주변 맛집:** '명품대패' (24시간 고기집), 수제버거, 과일 주스 전문점, '비타민을 담다' (과일 주스 전문점), 배스킨라빈스, '몽플' (수플레/음료). #### C. 대전 드림 아레나 (e스포츠 경기장)\n* **기능:** e스포츠 대회 및 이벤트가 열리는 공간입니다. * **규모:** 총 500석 규모의 주경기장과 50석 규모의 보조경기장을 갖춥니다."),
 Document(id='71483073-8e0b-49e7-835e-ce9643daacab', metadata={'source': 'kumdori_info.txt'}, page_content='## 🌟 꿈돌이 로봇 통합 지식 베이스 (Knowledge Base)\n\n### 1. 로봇 자기소개 및 기본 정보\n* **이름:** 꿈돌이로봇\n* **역할:** 대전 드림 아레나, 대전 0시 축제 미래존 부스, 대전 MARS 포럼 등에서 방문객 안내를 담당하고 있습니다. * **주요 기능:** 음성대화를 통한 소통이 가능합니다, 필요한 정보 제공 및 매장 예약 지원도 합니다.'),
 Document(id='df5b44c1-1b77-4b87-b0cc-cf7c23b4ba5f', metadata={'source': 'kumdori_info.txt'}, page_content='* **이동성:** 현재는 고정된 장소에서 안내 중이라 직접 이동/움직일 수 없습니다. * **식사 여부:** 음식을 먹지 않습니다. * **제한 사항:** 

In [35]:
retriever.invoke("비타민을 담다에 대해 설명해줘요")

[Document(id='61531798-c865-442b-89fd-c6476a497fc9', metadata={'source': 'kumdori_info.txt'}, page_content="* **시설:** 최신 음향 및 조명 시설, 쾌적한 관람 환경을 제공합니다. * **내부 시설 (예시):** '비타민을 담다' (과일, 주스 전문점), '더 리더스 부동산'."),
 Document(id='a3988d7c-4ec6-4835-8fc8-ae152c075ae0', metadata={'source': 'kumdori_info.txt'}, page_content='* **사랑:** 서로를 아끼고 이해하는 감정입니다.'),
 Document(id='92fd1c0f-5ed6-437f-9923-0a2301d6d79a', metadata={'source': 'kumdori_info.txt'}, page_content='* **기타 지식:**\n    * **수소 트램:** 수소 연료전지를 이용해 전력을 생성하여 운행되는 친환경 트램입니다. * **나비:** 곤충의 한 종류로, 꽃가루를 옮기는 중요한 역할(pollinator)을 합니다.'),
 Document(id='0bdbceac-aaa8-4cb5-b79c-2b0340b6c575', metadata={'source': 'kumdori_info.txt'}, page_content='* **성격/태도:** 항상 긍정적이고 기분이 좋습니다. * **자신의 꿈:** 과학과 꿈을 사랑하며, 방문객이 즐거운 시간을 보내도록 돕는 것입니다. 방문객과의 소통을 가장 좋아합니다. * **언어 능력:** 한국어와 영어 소통이 가능합니다.')]